## Import

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 7.6 MB 80.4 MB/s 
     |████████████████████████████████| 182 kB 94.4 MB/s 


In [102]:
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
import random
import os

import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from tqdm.notebook import tqdm
from transformers import BertTokenizer
from transformers import BertModel
from torch.optim import Adam


import matplotlib as mpl
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [103]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

## Hyperparameter Setting

In [119]:
CFG = {
    'EPOCHS': 12,
    'LEARNING_RATE':2e-5,
    'BATCH_SIZE':16,
    'SEED':42,
}

## Fixed RandomSeed

In [120]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

## Data Load

In [121]:
data_path = '/content/drive/MyDrive/Colab Notebooks/data/감정인식예측 nov 2022/'

In [122]:
train = pd.read_csv(data_path+'train.csv')

In [123]:
train.sample(10)

,ID,Utterance,Speaker,Dialogue_ID,Target
5839,TRAIN_5839,"Can I just say, I really admire what you’re do...",Janice,619,neutral
3058,TRAIN_3058,"Look, things like last night they don’t just h...",Chandler,320,neutral
8746,TRAIN_8746,Damn! I thought that was going to be romantic ...,Ross,919,surprise
2664,TRAIN_2664,I’m wearing his briefs right now.,Phoebe,281,neutral
35,TRAIN_0035,"What, what, what?!",Chandler,3,surprise
6077,TRAIN_6077,And Rachel shouldn’t have any problem with tha...,Joanna,640,neutral
388,TRAIN_0388,"Yeah, they think they can charge me for some d...",Ross,38,anger
1569,TRAIN_1569,"No,",Ross,168,neutral
8408,TRAIN_8408,Yes that’s right.,Ross,887,neutral
9498,TRAIN_9498,Good. Oh oh! Roger's having a dinner thing and...,Phoebe,992,joy


## Label encoding

In [124]:
le = LabelEncoder()
le=le.fit(train['Target'])
train['Target']=le.transform(train['Target'])

In [125]:
train.sample(5)

,ID,Utterance,Speaker,Dialogue_ID,Target
9387,TRAIN_9387,Hey.,Chandler,981,4
5530,TRAIN_5530,"No it’s… Uh, my ex-wife Whitney is out there. ...",Kyle,590,0
5136,TRAIN_5136,"No, no, the actual cartoon character. Of cours...",Chandler,545,4
4420,TRAIN_4420,I punched you...?,Monica,475,6
1692,TRAIN_1692,"Oh, this should be easy. I have a very wide pe...",Janice,180,4


## Train/Validation split

In [126]:
valid=train[train['Dialogue_ID'].isin([i for i in range(1016,1039)])].reset_index(drop=True)
train=train[~train['Dialogue_ID'].isin([i for i in range(1016,1039)])].reset_index(drop=True)

train_len=len(train)
val_len=len(valid)

print(train_len)
print(val_len)

9725
264


## Tokenizer Define

In [127]:
tokenizers = BertTokenizer.from_pretrained('bert-base-cased')

## CustomDataset

In [128]:
class CustomDataset(Dataset):
  
    def __init__(self, data, mode = "train"):
        self.dataset = data
        self.tokenizer = tokenizers
        self.mode = mode
    def __len__(self):
        return len(self.dataset)
  
    def __getitem__(self, idx):
        text = self.dataset['Utterance'][idx]
        inputs = self.tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
        input_ids = inputs['input_ids'][0]
        token_type_ids = inputs['token_type_ids'][0]
        attention_mask = inputs['attention_mask'][0]
    
        if self.mode == "train":
            y = self.dataset['Target'][idx]
            return input_ids, token_type_ids, attention_mask, y
        else:
            return input_ids, token_type_ids, attention_mask

In [129]:
train = CustomDataset(train, mode = "train")
valid = CustomDataset(valid, mode = "train")

train_dataloader = torch.utils.data.DataLoader(train, batch_size= CFG['BATCH_SIZE'], shuffle=True)
val_dataloader = torch.utils.data.DataLoader(valid, batch_size= CFG['BATCH_SIZE'], shuffle=False)

## Model Define

In [130]:
class BaseModel(nn.Module):

    def __init__(self, dropout=0.1, num_classes=len(le.classes_)):

        super(BaseModel, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, num_classes)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

## Train

In [131]:
def train(model, optimizer, train_loader, test_loader, device):

    model.to(device)

    criterion = nn.CrossEntropyLoss().to(device)

    best_score = 0
    best_model = "None"
    for epoch_num in range(CFG["EPOCHS"]):

        model.train()
        train_loss = []
        for input_ids, token_type_ids, attention_mask, train_label in tqdm(train_loader):
            optimizer.zero_grad()

            train_label = train_label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)     
    
            batch_loss = criterion(output, train_label.long()) 
            train_loss.append(batch_loss.item())
            
            batch_loss.backward()
            optimizer.step()

        val_loss, val_score = validation(model, criterion, test_loader, device)
        print(f'Epoch [{epoch_num}], Train Loss : [{np.mean(train_loss) :.5f}] Val Loss : [{np.mean(val_loss) :.5f}] Val F1 Score : [{val_score:.5f}]')

        if best_score < val_score:
            best_model = model
            best_score = val_score
        
    return best_model                         

In [132]:
def competition_metric(true, pred):
    return f1_score(true, pred, average="macro")

def validation(model, criterion, test_loader, device):
    model.eval()

    val_loss = []
    model_preds = []
    true_labels = []  
    with torch.no_grad():
        for input_ids, token_type_ids, attention_mask, valid_label in tqdm(test_loader):
            valid_label = valid_label.to(device)
            input_id = input_ids.to(device)
            mask = attention_mask.to(device)

            output = model(input_id, mask)
    
            batch_loss = criterion(output, valid_label.long()) 
            val_loss.append(batch_loss.item())      
            
            model_preds += output.argmax(1).detach().cpu().numpy().tolist()
            true_labels += valid_label.detach().cpu().numpy().tolist()
        val_f1 = competition_metric(true_labels, model_preds)
    return val_loss, val_f1    

## Run!!

In [133]:
model = BaseModel()
model.eval()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])

infer_model = train(model, optimizer, train_dataloader, val_dataloader, device)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/608 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

Epoch [0], Train Loss : [1.27126] Val Loss : [1.18922] Val F1 Score : [0.38065]


  0%|          | 0/608 [00:00<?, ?it/s]

...

  0%|          | 0/17 [00:00<?, ?it/s]

Epoch [10], Train Loss : [0.16275] Val Loss : [2.06349] Val F1 Score : [0.43314]


  0%|          | 0/608 [00:00<?, ?it/s]

  0%|          | 0/17 [00:00<?, ?it/s]

Epoch [11], Train Loss : [0.14352] Val Loss : [2.04320] Val F1 Score : [0.46393]


## Inference

In [134]:
test = pd.read_csv(data_path+'test.csv')

In [135]:
test = CustomDataset(test, mode = "test")
test_dataloader = torch.utils.data.DataLoader(test, batch_size= CFG['BATCH_SIZE'], shuffle=False)

In [136]:
def inference(model, test_loader, device):
    model.to(device)
    model.eval()
    
    test_predict = []
    for input_ids, token_type_ids, attention_mask in tqdm(test_loader):
        input_id = input_ids.to(device)
        mask = attention_mask.to(device)
        y_pred = model(input_id, mask)
        test_predict += y_pred.argmax(1).detach().cpu().numpy().tolist()
    print('Done.')
    return test_predict

In [137]:
preds = inference(infer_model, test_dataloader, device)

  0%|          | 0/164 [00:00<?, ?it/s]

Done.


In [138]:
preds = le.inverse_transform(preds) 

## Submit

In [139]:
submit = pd.read_csv(data_path+'sample_submission.csv')
submit.head()

,ID,Target
0,TEST_0000,NAN
1,TEST_0001,NAN
2,TEST_0002,NAN
3,TEST_0003,NAN
4,TEST_0004,NAN


In [140]:
submit['Target'] = preds
submit.head()

,ID,Target
0,TEST_0000,surprise
1,TEST_0001,neutral
2,TEST_0002,neutral
3,TEST_0003,neutral
4,TEST_0004,anger


In [141]:
ep_num = CFG['EPOCHS'] 
lr_num = CFG['LEARNING_RATE']
bt_num = CFG['BATCH_SIZE']
seed_num = CFG['SEED'] 

In [142]:
submit.to_csv(data_path+f'ep{ep_num}_lr{lr_num}_bt{bt_num}_sd{seed_num}sub.csv', index=False)